In [1]:
# jupyter-notebook V0_1_1.ipynb --port 8888 --ip 192.168.1.5

import cv2
import os
import numpy as np
from collections import Counter
import itertools  
from collections import defaultdict
import time
import numba as nb
import copy
from os import listdir, mkdir
from os.path import isfile, isdir, join

from components_container import ComponentsContainer

%matplotlib inline

In [2]:
# filter all noise using minimum size threshold
BACKGROUND_INTENSITY_THRESHOLD = 20
MIN_AXON_SIZE = 20

# merging close components based on centroids and contour distance
PREMERGE_CENTROIDS_DISTANCE_T  = 200
PREMERGE_CONTOUR_DISTANCE_T = 15

#splitting nucleus and axons based on size
MIN_NUCLEUS_SIZE = 150

#grouping possible axons to nucleus
CENTROIDS_DISTANCE_T = 90
CONTOUR_DISTANCE_T = 25

LOADPATH = '../raw_pics'
SAVEPATH = '../info'
RECURSIVE = True

In [23]:
def mirror_dir_for_save(loadpath):
    try:
        mkdir(loadpath.replace(LOADPATH, SAVEPATH))
    except(OSError):
        pass
    
def process_dir(loadpath):
    mirror_dir_for_save(loadpath)
    object_names_in_folder = listdir(loadpath)
    print ("start processing folder {} pics: {}".format(loadpath, str(len(object_names_in_folder))))
    for f in object_names_in_folder:
        local_object = join(loadpath, f)

        if isfile(local_object) and f[0] != '.':
            process_pic(join(loadpath, f))
            continue

        if RECURSIVE and isdir(join(loadpath, f)):
            process_dir(join(loadpath, f))
            continue

def process_pic(path):
    starttime = time.time()
    print("processing pic {} ... ".format(path), end = "")
    img = cv2.imread(path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    ret, binary = cv2.threshold(gray, BACKGROUND_INTENSITY_THRESHOLD, 400, cv2.THRESH_BINARY)
    container = ComponentsContainer(binary, MIN_AXON_SIZE)
    container.merge_components_closer_than(PREMERGE_CENTROIDS_DISTANCE_T, PREMERGE_CONTOUR_DISTANCE_T)
    container.recalc_edge_and_size_info()    
    container.split_nucl_axon(MIN_NUCLEUS_SIZE)
    container.add_hull_info_to_nucleus()
    container.group_axons_to_nucleus(CENTROIDS_DISTANCE_T, CONTOUR_DISTANCE_T)
    with open(path.replace(LOADPATH, SAVEPATH).replace(".bmp", ".txt"), "w") as f:
        write_info_from_container(container, f)
    save_pic_from_container(container,img, path.replace(LOADPATH, SAVEPATH).replace(".bmp", "_processed.png"))
    print('done in {} sec.'.format(time.time() - starttime))


def write_info_from_container(container, f):
    labs = sorted(container.nucleus_labs, key = lambda x: -container.components_index.get(x).size)
    f.write("#, Size, HullSize, Perimeter\n")
    i = 0
    for nucl_lab in labs:
        i +=1
        nucl = container.components_index.get(nucl_lab)
        f.write("{:2}, {:4}, {:4}, {:4}\n".format(i, nucl.size, nucl.hull_size, len(nucl.edge)))
        
def save_pic_from_container(container,img, savepath):
    pic = enhance_img_with_container(img, container)
    cv2.imwrite(savepath,pic)
    
def enhance_img_with_container(img, container):
    edge_color = [80,0,80]
    hull_color = [0,0,100]
    for nucl_lab in container.nucleus_labs:           
        hull = np.array(container.components_index.get(nucl_lab).hull, np.int32)
        hull.reshape((-1,1,2))
        hull = hull[:,::-1]
        cv2.polylines(img, [hull],True, hull_color)

        for ed in container.components_index.get(nucl_lab).edge:
            img[ed[0], ed[1]] = edge_color
    return img

In [ ]:
#LOADPATH = '../raw_pics/hippo/с4_6_73'
process_dir(LOADPATH)

start processing folder ../raw_pics pics: 5
start processing folder ../raw_pics/.ipynb_checkpoints pics: 0
start processing folder ../raw_pics/hippo pics: 6
start processing folder ../raw_pics/hippo/int_56 pics: 21
processing pic ../raw_pics/hippo/int_56/int_56_hippo_1.bmp ... done in 42.47242712974548 sec.
processing pic ../raw_pics/hippo/int_56/int_56_hippo_10.bmp ... done in 74.86228585243225 sec.
processing pic ../raw_pics/hippo/int_56/int_56_hippo_11.bmp ... done in 124.4344220161438 sec.
processing pic ../raw_pics/hippo/int_56/int_56_hippo_12.bmp ... done in 52.0018630027771 sec.
processing pic ../raw_pics/hippo/int_56/int_56_hippo_13.bmp ... done in 84.4428243637085 sec.
processing pic ../raw_pics/hippo/int_56/int_56_hippo_14.bmp ... done in 87.71539092063904 sec.
processing pic ../raw_pics/hippo/int_56/int_56_hippo_15.bmp ... done in 69.1942069530487 sec.
processing pic ../raw_pics/hippo/int_56/int_56_hippo_16.bmp ... done in 62.25274610519409 sec.
processing pic ../raw_pics/hi

In [8]:
# testing

img = cv2.imread('../raw_pics/ssc/int/74/int74_iba_cortex__left_03z.bmp')
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
ret, binary = cv2.threshold(gray, BACKGROUND_INTENSITY_THRESHOLD, 400, cv2.THRESH_BINARY)
container = ComponentsContainer(binary, MIN_AXON_SIZE)
container.merge_components_closer_than(PREMERGE_CENTROIDS_DISTANCE_T, PREMERGE_CONTOUR_DISTANCE_T)
container.recalc_edge_and_size_info()    
container.split_nucl_axon(MIN_NUCLEUS_SIZE)
container.add_hull_info_to_nucleus()
container.group_axons_to_nucleus(CENTROIDS_DISTANCE_T, CONTOUR_DISTANCE_T)

In [22]:
pic = enhance_img_with_container(img, container)
cv2.imwrite('../test.png',pic)


True